In [1]:
#Iris Localization:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

def irislocalization(image):
    #image_gray
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    _,image_binary = cv2.threshold(image,60,65,cv2.THRESH_BINARY)
    
    #find the darket point - pupil
    Yp = np.argmin(image_binary.mean(axis = 1))
    Xp = np.argmin(image_binary.mean(axis = 0))

    #Crop the image to smaller sizes
    crop_img = image[max(0,Yp-100):min(Yp+100,320), max(0,Xp-100):min(Xp+100,280)]
    
    #Pupil Area
    area120 = image[max(0, Yp-60):min(Yp+60, 320), max(0, Xp-60):min(Xp+60, 280)]
    
    mask1 = np.where(image_binary>0,1,0)

    miny = np.argmin(mask1.mean(axis = 1))
    minx = np.argmin(mask1.mean(axis = 0))

    radius1 = np.sum(mask1[miny] == 0) / 2
    radius2 = np.sum(mask1 == 0,axis=0)[minx] / 2
    radius = np.mean([radius1, radius2])
        
    crop_img_copy2 = crop_img.copy()
    
    #Find the inner circles, some times when the radius range is too small it fails to draw the inner circle
    for i in range(1,7):
        circles_inner = cv2.HoughCircles(area120,cv2.HOUGH_GRADIENT,1,250,
                            param1=50,param2=10,minRadius=int(radius)-i,
                                 maxRadius=int(radius)+i)
        if type(circles_inner) == type(None):
            pass
        else:
            break
        
    circles_inner = np.uint16(np.around(circles_inner))
    
    
    #Find the outer circle
    circles_outer = cv2.HoughCircles(crop_img,cv2.HOUGH_GRADIENT,1,250,
                            param1=40,param2=10,minRadius=int(radius)+15, maxRadius=118)

    circles_outer = np.uint16(np.around(circles_outer))

    #Draw Inner
    for i in circles_inner[0,:]:
        cv2.circle(crop_img_copy2,(int(i[0]+40),int(i[1]+40)),int(i[2]),(0,255,0),2) #plus 40 to map to the crop_img
        inner_circle = [int(i[0]+40),int(i[1]+40),int(i[2])]

    #Draw Outer
    for i in circles_outer[0,:]:
        cv2.circle(crop_img_copy2,(i[0],i[1]),i[2],(0,255,0),2)
        outer_circle = [i[0],i[1],i[2]]
    
    return (crop_img, inner_circle, outer_circle)

In [2]:
#Iris Normalization:
def irisnormalization(crop_img, inner_circle, outer_circle):
    M = 64
    N = 512
    
    row,column = crop_img.shape
    
    Image_norm = np.zeros((64,512))

    [x1, y1, r_inner] = [inner_circle[0], inner_circle[1], inner_circle[2]]
    [x2, y2, r_outer] = [outer_circle[0], outer_circle[1], outer_circle[2]]

    #distance between the center of pupil to center of the iris
    distance = np.sqrt((x1-x2)**2 + (y1-y2)**2)

    #angle between the center of pupil to center of the iris
    theta_diff = np.arctan2(y2-y1, x2-x1)

    #using law of cosines to find the distance between the center of pupil to the outer point
    diff_dist_square = distance**2 + r_outer**2 - 2*distance*r_outer*np.cos(theta_diff)
    diff_dist = diff_dist_square**0.5
    
    #Using the centroid of pupil as the reference point
    for Y in range(0, M):
        for X in range(0, N):
            theta = 2*np.pi*X/N
            x_inner = x1+r_inner*np.cos(theta)
            y_inner = y1+r_inner*np.sin(theta)
            x_outer = x1 + diff_dist * np.cos(theta)
            y_outer = y1 + diff_dist * np.sin(theta)
            x = x_inner + (x_outer - x_inner)*Y/M
            x = int(x)
            x = min(column-1,x) or max(0,x)
            y = y_inner + (y_outer - y_inner)*Y/M
            y = int(y)
            y = min(row-1,y) or max(0,y)
            Image_norm[Y, 511-X] = crop_img[y,x]
            
    return Image_norm

In [3]:
#Image Enhancement
def imgenhancement(Image_norm):
    
    image2en = np.array(Image_norm,dtype=np.uint8)
    img_histEql = cv2.equalizeHist(image2en)

    #Detection of reflections and eyelashes, then use as a mask to do logic operation
    _,image_noises = cv2.threshold(Image_norm, 178,255, cv2.THRESH_BINARY)
    image_noises = np.array(image_noises,dtype=np.uint8)
    #plt.imshow(image_reflection, cmap = 'gray')

    #masks:
    mask1 = cv2.bitwise_not(image_noises)
    mask1 = np.array(mask1,dtype=np.uint8)
    #image_reflection = np.array(image_reflection,dtype=np.uint8)

    noise_removed = cv2.bitwise_and(img_histEql,img_histEql, mask = mask1)
    
    return noise_removed

In [4]:
#Feature Extraction:

import scipy.signal

# how should we determine the frequency value
sigx_1 = 3
sigy_1 = 1.5
sigx_2 = 4.5
sigy_2 = 1.5
f = 1/sigy_1   # how should we determine the frequency value

kernel_1 = np.zeros((9,9))
kernel_2 = np.zeros((9,9))

#The modulating function for the defined filter
def mod_defined(x,y,f):
    return np.cos(2*np.pi*f*np.sqrt(x**2 + y**2))

#The gabor filter function
def G(x,y,sigx,sigy):
    a = 1/(2*np.pi*sigx*sigy)
    b = np.exp(-0.5*(x**2/sigx**2 + y**2/sigy**2))*mod_defined(x,y,f)
    return a*b

#getting the filters for 2 channels
for row in range(9):
    for col in range(9):
        kernel_1[row,col] = G(col,row,sigx_1,sigy_1)
        kernel_2[row,col] = G(col,row,sigx_2,sigy_2)
        
#Getting the feature vectors for the filtered two images
def feature_vector(noise_removed):
    roi = noise_removed[0:48,:]
    vector = []
    block_horizontal = int(len(roi[0])/8)
    block_vertical = int(len(roi)/8)
    for i in range(block_horizontal):
        for j in range(block_vertical): #specified the the block
            roi_block = roi[j*8:(j+1)*8,i*8:(i+1)*8]
            filtered_roi_1 = scipy.signal.convolve2d(roi_block, kernel_1, mode = 'same')
            filtered_roi_1 = np.absolute(filtered_roi_1) #as stated in the paper, use absolute values to calculate mean and std
            mean = filtered_roi_1.mean()
            vector.append(mean)
            stdev = filtered_roi_1.std()
            vector.append(stdev)
            
    #for the second channel        
    for i in range(block_horizontal):
        for j in range(block_vertical): #specified the the block
            roi_block = roi[j*8:(j+1)*8,i*8:(i+1)*8]
            filtered_roi_2 = scipy.signal.convolve2d(roi_block, kernel_2, mode = 'same')
            filtered_roi_2 = np.absolute(filtered_roi_2)
            mean = filtered_roi_2.mean()
            vector.append(mean)
            stdev = filtered_roi_2.std()
            vector.append(stdev)
            
    return vector 

In [5]:
#rotations
def rotation(image):
    templates = []
    #degrees = [-9,-6,-3,0,3,6,9]  #of initial image
    degrees = [-3,-2,-1,0,1,2,3]
    for i in degrees:
        loc_norm = int(512*i/360)
        if i > 0:
            rotated = np.hstack([image[:,loc_norm:],image[:,:loc_norm]])
        else:  #case when loc norm < 0
            rotated = np.hstack([image[:,(512 + int(loc_norm)):],image[:,:(512 + int(loc_norm))]] )
        
        templates.append(rotated)
    
    return templates

In [6]:
# trainning_data has rotations

def trainning():
    vec_4_all = []
    vec_4_set = []
    vec_4_pic = []
    
    file_main_Path = '/Users/sunflower/Desktop/GR5293_Image_Analysis/iris/'#001/1/
    sets = []
    #try to get the image sets and find the right path
    for i in range(1,109):
        data = '00' + str(i)
        if len(data) == 4:
            data = data[1:]
        if len(data) == 5:
            data = data[2:]
        else:
            data
        sets.append(data) 
    sets = sets
    
    for img_set in sets:
        #print(img_set)
        filePath_set = file_main_Path + img_set
        
        for image_num in range(1,4):
            num = str(image_num)
            img_name = img_set + '_' + '1' + '_' + num +'.bmp'
            folder_image = '/1/'+ img_name
            filePath = filePath_set + '/1/'+ img_name
            #print(filePath)
            img = cv2.imread(filePath)
            folder_image =''
            
            crop_img, inner_circle, outer_circle = irislocalization(img)
            image_norm = irisnormalization(crop_img, inner_circle, outer_circle)
            templates = rotation(image_norm)
            
            for each_tem in templates:
                noise_removed = imgenhancement(each_tem)
                enhanced_img = imgenhancement(noise_removed)
                feature = feature_vector(noise_removed)
                vec_4_pic.append(feature)
            
            #vec_4_set.append(vec_4_pic)
            
       # vec_4_all.append(vec_4_set)
        
    return vec_4_pic    

In [7]:
def testing():
    vec_4_all = []
    vec_4_set = []
    
    file_main_Path = '/Users/sunflower/Desktop/GR5293_Image_Analysis/iris/'#001/1/
    sets = []
    #try to get the image sets and find the right path
    for i in range(1,109):
        data = '00' + str(i)
        if len(data) == 4:
            data = data[1:]
        if len(data) == 5:
            data = data[2:]
        else:
            data
        sets.append(data) 
    sets = sets
    
    for img_set in sets:
        #print(img_set)
        filePath_set = file_main_Path + img_set
        
        for image_num in range(1,5):
            num = str(image_num)
            img_name = img_set + '_' + '2' + '_' + num +'.bmp'
            folder_image = '/2/'+ img_name
            filePath = filePath_set + '/2/'+ img_name
            #print(filePath)
            img = cv2.imread(filePath)
            
            crop_img, inner_circle, outer_circle = irislocalization(img)
            image_norm = irisnormalization(crop_img, inner_circle, outer_circle)
            noise_removed = imgenhancement(image_norm)
            enhanced_img = imgenhancement(noise_removed)
            feature = feature_vector(noise_removed)
            
            folder_image =''
            vec_4_set.append(feature)
            
        #vec_4_all.append(vec_4_set)
        
    return vec_4_set    

In [8]:
trainingdata = trainning()

In [9]:
testingdata = testing()

In [387]:
#iris matching:
#return accuracy rate, L1,L2,Cosine distance for table
from scipy.spatial import distance
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

#distance.euclidean : L1
#distance.sqeuclidean : L2
#distance.cosine : Cosine
def irismatching(trainingdata, testingdata, dimension):

    trainingdata = np.array(trainingdata)
    trainingdata = np.where(np.isnan(trainingdata), 0, trainingdata)

    testingdata = np.array(testingdata)
    testingdata = np.where(np.isnan(testingdata), 0, testingdata)

    sets = np.arange(1,109)

    temp_train_match_to = np.repeat(sets, 21)

    train_match_to = np.repeat(sets,3) #subject to change

    testing_matching_to = np.repeat(sets, 4) #subject to change

#model
    clf = LDA(n_components = dimension)
    clf.fit(trainingdata, temp_train_match_to)  #only takes <=2 dimension data

#LDA
    training_reduced = clf.transform(trainingdata)
    testing_reduced = clf.transform(testingdata)

    predict_result = []                    #predict picture beglong to which set
    L1_predict_result = []
    L2_predict_result = []
    
    for st in range(np.shape(testingdata)[0]):  #for each testing picture
        shortest_distances = []                 #to find the each training picture that has the shortest distance with it
        L1_shortest_distances =[]
        L2_shortest_distances = []
        
        test_lda = testing_reduced[st]          #corresponding LDA of the vector

        for pic in range(int(np.shape(trainingdata)[0]/7)):    #each picture in the training set
            cos_distances = []                          #cos distances of each templates for the picture
            L1_distances = []
            L2_distances = []
            
            for tem in range(7):           #calculated the distiance between feature vector of picture set and a picture template
                L1 = distance.euclidean(test_lda, training_reduced[pic*7+tem])
                L2 = distance.sqeuclidean(test_lda,training_reduced[pic*7+tem])
                cos = distance.cosine(test_lda, training_reduced[pic*7+tem])

                cos_distances.append(cos)   #distances from each templates
                L1_distances.append(L1)
                L2_distances.append(L2)
                
            shortest_distances.append(min(cos_distances))  #get the shortest distances among templates; use as picture distance
            L1_shortest_distances.append(min(L1_distances))
            L2_shortest_distances.append(min(L2_distances))
            
        shortest_dist_loc = np.argmin(shortest_distances)  #most similar pictures
        L1_shortest_dist_loc = np.argmin(L1_shortest_distances)
        L2_shortest_dist_loc = np.argmin(L2_shortest_distances)
        
        predict_result.append(train_match_to[shortest_dist_loc])
        L1_predict_result.append(train_match_to[L1_shortest_dist_loc])
        L2_predict_result.append(train_match_to[L2_shortest_dist_loc])
        
    predict_result = np.array(predict_result, dtype = np.int)
    L1_predict_result = np.array(L1_predict_result, dtype = np.int)
    L2_predict_result = np.array(L2_predict_result, dtype = np.int)
    
    cos_accuracy = 1-sum(predict_result != testing_matching_to)/len(testing_matching_to)
    L1_accuracy = 1-sum(L1_predict_result != testing_matching_to)/len(testing_matching_to)
    L2_accuracy = 1-sum(L2_predict_result != testing_matching_to)/len(testing_matching_to)
    
    return cos_accuracy, L1_accuracy, L2_accuracy, Predict_result

In [388]:
#Performance Evaluation:
import pandas

def CRR_Curve(training, testing):
    
    dimension = [50, 60, 70, 80, 90, 100, 107]
    rate = np.zeros(7)
    rate_L1 = np.zeros(7)
    rate_L2 = np.zeros(7)
    for i in range(7):
        cos_accuracy, L1_accuracy, L2_accuracy, _, = irismatching(training, testing, dimension[i])
        rate[i] = cos_accuracy
        rate_L1[i] = L1_accuracy
        rate_L2[i] = L2_accuracy
        
    plt.subplot(1,3,1)   
    plt.plot(dimension, rate)
    plt.title('CRR of cosine distance measure')
    plt.xlabel('Dimensionality of the feature vector')
    plt.ylabel('correct recognition rate')
    
    plt.subplot(1,3,2)
    plt.plot(dimension, rate_L1)
    plt.title('CRR of L1 distance measure')
    plt.xlabel('Dimensionality of the feature vector')
    plt.ylabel('correct recognition rate')
    
    plt.subplot(1,3,3)
    plt.plot(dimension, rate_L2)
    plt.title('CRR of L2 distance measure')
    plt.xlabel('Dimensionality of the feature vector')
    plt.ylabel('correct recognition rate')
    
    plt.show

def similarity_measure_table(training, testing):
    
    cos_1, L1_1, L2_1, _, = irismatching(training, testing, 107)
    
    cos, L1, L2, _, = irimatching(training, testing, 100)
    
    table_data = [[L1_1,L1],[L2_1,L2],[cos_1,cos]]
    
    table_data = pandas.DataFrame(table_data)
    
    table_data.index = ['L1 Distance Measure', 'L2 Distance Measure', 'Cosine Distance Measure']
    table_data.columns = ['Original Feature Set', 'Reduced Feature Set']
    
    print(table_data)
    
#def falsematch(training, testing):

#def falsenonmatch(training, testing):

In [41]:
#iris matching for roc:
#return accuracy rate, L1,L2,Cosine distance for table
from scipy.spatial import distance
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

#distance.euclidean : L1
#distance.sqeuclidean : L2
#distance.cosine : Cosine

def matching_roc(trainingdata, testingdata, dimension):
    trainingdata = np.array(trainingdata)
    trainingdata = np.where(np.isnan(trainingdata), 0, trainingdata)

    testingdata = np.array(testingdata)
    testingdata = np.where(np.isnan(testingdata), 0, testingdata)

    sets = np.arange(1,109)

    temp_train_match_to = np.repeat(sets, 21)

    train_match_to = np.repeat(sets,3) #subject to change

    testing_matching_to = np.repeat(sets, 4) #subject to change

    #model
    dimension = 107
    clf = LDA(n_components = dimension)
    clf.fit(trainingdata, temp_train_match_to)  #only takes <=2 dimension data
    #clf.score(testingdata, testing_matching_to)
    prediction = clf.predict(testingdata)
    y_scores = clf.predict_proba(testingdata)[:,1]
    return testing_matching_to, prediction, y_scores

def new_prediction(actual, prediction, y_scores, threshold):
   
    FalseP = 0
    FalseN = 0
    
    new = []
    for i in range(len(y_scores)):
        if y_scores[i] < threshold:
            new.append(0)    #non-match
        else:
            new.append(1)    #accepted
    
    for i in range(len(y_scores)):
        if actual[i] != prediction[i] and new[i] == 1: #actual != prediction but accepted
            FalseP +=1
        if actual[i] == prediction[i] and new[i] == 0: #actual == prediction but threshollded to be non-matching
            FalseN +=1
    
    TotalP = sum(new)
    TotalN = len(new) - sum(new)
    
    FMR = FalseP/TotalP
    FRR = FalseN/TotalN
    return FMR, FRR


def get_roc(trainging, testing):
    dimension = 107
    testing_matching_to, prediction, y_scores = matching_roc(trainingdata, testingdata, dimension)
    
    threshold = [0.446, 0.472, 0.502]
    
    table  = []
    for i in threshold:
        FMR, FRR = new_prediction(testing_matching_to, prediction, y_scores, i)
        table.append([i,FMR,FRR])
        
    table = pandas.DataFrame(table)
    table.columns = ['Threshold', 'False Match Rate', 'False Non Match Rate']
    
    print(table)

In [43]:
import pandas
get_roc(trainingdata, testingdata)

   Threshold  False Match Rate  False Non Match Rate
0      0.446               0.0              0.759907
1      0.472               0.0              0.759907
2      0.502               0.0              0.759907


In [389]:
similarity_measure_table(trainingdata, testingdata)

                         Original Feature Set  Reduced Feature Set
L1 Distance Measure                  0.768519             0.761574
L2 Distance Measure                  0.768519             0.761574
Cosine Distance Measure              0.803241             0.789352


In [ ]:
def compute_falses(dist_mtx, threshold):  # dist_mtx each row is the dist of 1 test img to all train imgs
    gt_acceptance = np.zeros_like(dist_mtx)
    for r in range(len(gt_acceptance)):
        idx = r // 4
        gt_acceptance[r][idx * 3: (idx + 1) * 3] = 1
    accept = dist_mtx <= threshold
    false_accept = (1 - gt_acceptance) * accept
    false_reject = gt_acceptance * (1 - accept)
    FAE = false_accept.sum() / (dist_mtx.shape[0] * dist_mtx.shape[1])
    FRE = false_reject.sum() / (dist_mtx.shape[0] * dist_mtx.shape[1])
    return FAE, FRE


def plot_roc(dist_mtx):
    thresholds = np.linspace(dist_mtx.min(), dist_mtx.max(), 50)

    FAEs = []
    FREs = []

    for t in thresholds:
        a, b = compute_falses(dist_mtx, t)
        FAEs.append(a)
        FREs.append(b)

    roc_table = pd.DataFrame({"Threshold": thresholds,
                              "FAE": FAEs,
                              "FRE": FREs})
    roc_table.to_csv("ROCTable.csv")

    plt.plot(np.array(FAEs), np.array(FREs), color='navy', linestyle='-')
    plt.xlabel('False Match Rate')
    plt.ylabel('False Non_match Rate')
    plt.title('FMR ROC Curve')